# 25. 더 많이! 더 깔끔하게! 데이터를 관리하는 데이터베이스

## 1. 들어가며
데이터는 보통 어떻게 저장해 사용할까?

기초 파이썬을 공부할 때를 기억해보자. 메모리에 올려 변수로 가리키는 데이터는 그 쓰임에 따라 number, string 형태로 저장하고, 배열이 필요할 때는 list 타입 또는 NumPy의 ndarray 타입으로 저장했다. 그리고 표와 같은 데이터는 딕셔너리나 Pandas의 Series, DataFrame의 데이터 타입을 사용했다.

이렇게 메모리에 저장된 데이터는 연산이 매우 빠르게 처리된다. 하지만 아쉽게도 영구적으로 저장되진 않는다. 데이터를 영구적으로 보존하기 위해 우리는 하드디스크에 파일 형태로 저장해야 한다. 데이터를 처리하는 프로세스가 종료된 후에도 데이터는 보존되었다가 재활용될 수 있어야 한다는 것을 데이터가 가져야 할 영속성(Persistence)이라고 한다. 하지만 데이터 영속성을 위하여 파일로 저장하는 것만으로는 비효율적일 수 있다. 특히 많은 사람들이 그 데이터를 공유할 때가 그렇다. 그래서 우리는 데이터베이스를 필요로 하게 된다.

<img src="./image/db.png" alt="DB" />

### 학습 목표
---
* 데이터를 연산하는 방법에 대해 학습한다.
* Pandas, Class를 이용한 데이터 연산법을 연습한다.
* Database에 대해 학습하고, Python을 이용해 간단한 Database를 직접 사용해보자.

### 학습 내용
---
* 데이터 관리 프로그램 만들기
    - 파일 시스템 활용
    - 판다스와 csv 파일
    - 판다스의 유용한 기능들
    - 판다스 Transform 실전연습
* 데이터베이스
    - 다중 사용자 환경
    - 데이터베이스의 세계로
    - SQL
    - 파이썬 DB API

### 준비물
---
아직 경로를 생성하지 않았다면 터미널을 열고 디렉토리를 생성하자.

```bash
$ mkdir -p ~/AIFFEL/Fundamentals/F25_db/data_handling/data
```

사용하는 데이터이다. 압축을 풀어 파일들을 data 폴더로 이동시키자.

```bash
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/db_data.zip
$ mv db_data.zip  ~/AIFFEL/Fundamentals/F25_db/data_handling/data
$ cd  ~/AIFFEL/Fundamentals/F25_db/data_handling/data
$ unzip db_data.zip
```

## 2. 데이터 관리 프로그램 만들기 (1) 파일 시스템 활용
당신은 데이터 관리 어플리케이션을 제작하는 파이썬 개발자이다. 당신의 멋진 파이썬 실력으로 텍스트 파일에 담긴 데이터를 읽어 들여 필요한 정보를 출력하는 어플리케이션을 만들어 볼 생각이다.

### 1단계. 문제 설명
---
사원 정보 프로그램을 만든다고 하자. `employeefile.txt` 란 파일에는 사원들의 정보가 담긴 데이터가 있다. 당신이 할 일은 파일을 읽어 들여 월급을 계산하고, 월급이 가장 높은 사원을 찾는 프로그램이다.

<img src="./image/employee.png" alt="employee" />

### 2단계. summarize() 함수 그리고 의사코드
---
자, 코딩을 하기 전에 전체 프로그램을 어떻게 구성할지 대략적으로 생각해보자.

```python
def summarize():
    #파일 이름을 입력받는다.
    #파일을 읽기 모드로 열어 한 줄씩 파일을 읽는다. 
    #파일의 각 행의 데이터 속성을 갖는 인스턴스 객체(사원객체)를 만든다. 
      # 사전에 employee 클래스 설계 필요!
      # 사전에 인스턴스 사원 객체를 만드는 함수 필요!  
    #월급이 가장 높은 사원을 찾는 코드를 구현한다. 
    #결과를 출력한다. 
    pass  
```

### 3단계. Employee Class 설계
---
자 우선 Employee 클래스부터 설계해보자.

인스턴스 변수로 `name`, `division`, `jobgrade`, `years`, `basic`을 갖는 `Employee` 클래스를 만들어보자.

In [2]:
class Employee:
    def __init__ (self, name, division, jobgrade, years, basic):
        self.name = name
        self.division = division
        self.jobgrade = int(jobgrade)
        self.years = int(years)
        self.basic = float(basic)
# 텍스트 파일에서 불러온 각각의 데이터는 string으로 처리됩니다. 따라서 형 변환을 해주세요.

설계한 Employee 클래스를 사용해 그림을 보고 "안태희(AhnTaeHee)"란 사원의 정보를 인스턴스 객체로 만들어 employee 변수로 지정하자. 아직 텍스트 파일을 불러오지 말고, 각각의 매개변수에 들어갈 값을 직접 스트링(string)으로 하드코딩하자.

In [3]:
employee1 = Employee('Ahn TaeHee', 'AI Team', '3', '3', '300')

### 4단계. `make_employee()` 함수 선언
---
위에서 한 것처럼 텍스트 형식의 파일을 파싱해 `Employee` 클래스 인스턴스를 생성하고 싶다. 하지만 한 명이야 이렇게 직접 타이핑할 수 있지만 사원 수가 많으면 일일이 타이핑하기 힘들것이다. 그러니 함수를 만들어 보자. 파일을 한 줄 한 줄 읽은 뒤 데이터의 값을 사용하여 자동으로 인스턴스 객체를 만들어 주는 함수이다.

함수 이름은 `make_employee()`라고 정한다.

In [4]:
def make_employee(inputdata):
    name, division, jopgrade, year, basic = inputdata.split(',')
    return Employee(name, division, jopgrade, year, basic)

반복문을 사용해서 이런 식으로 사용할 것이다. 파일을 열어서 모든 라인을 순회하며 `make_employee()`를 호출하자.

In [5]:
import os
file_path = os.getenv("HOME") + '/AIFFEL/Fundamentals/F25_db/data_handling/data/employeefile.txt'
print(file_path)
inputfile = open(file_path, 'r')

for i in inputfile:
    e = make_employee(i)

/home/aiffel-dj10/AIFFEL/Fundamentals/F25_db/data_handling/data/employeefile.txt


### 5단계. Employee Class 이어서..
---
사실 `Employee` 클래스 설계는 끝난 게 아니다. `<인스턴스>.<속성값>` 과 같은 형식을 사용하여 인스턴스 변수에 접근할 수도 있지만, 보통은 메소드로 구현해 값을 반환받는 것이 일반적이다. 이는 Clean Code 운동과 관련된 내용이니 통상 이런 식으로 구현한다 정도로만 이해하면 된다.

#### 정보 접근자 메소드

In [6]:
class Employee:
    def get_name(self):
        return self.name
    def get_division(self):
        return self.division
    def get_jobgrade(self):
        return self.jobgrade
    def get_years(self):
        return self.years
    def get_basic(self):
        return self.basic

#### 급여 계산
급여는 기본 급여(basic salary)에, 기본 급여의 10% * 근속연수를 더한 값이다. 즉 급여(salary)는 `basic salary + years*basic*0.1 `과 같이 계산할 수 있다.

아래 코드 블록에 급여를 계산하는 `salary()` 메소드를 구현해보자.

In [7]:
def get_salary(self):
    self.salary = self.basic + (self.basic * self.years * 0.1)
    return self.salary

#### 완성된 Employee 클래스
필요한 메소드를 모두 구현한 클래스는 다음과 같다.

salary에 초깃값 0을 설정해 주었다.

In [8]:
class Employee:
    def __init__ (self, name, division, jobgrade, years, basic):
        self.name = name
        self.division = division
        self.jobgrade = int(jobgrade)
        self.years = int(years)
        self.basic = float(basic)
        self.salary = 0

    def get_name(self):
        return self.name

    def get_divison(self):
        return self.division
    
    def get_jobgrade(self):
        return self.jobgrade
    
    def get_years(self):
        return self.years
    
    def get_basic(self):
        return self.basic

    def get_salary(self):
        self.salary = self.basic + (self.basic * self.years * 0.1)
        return self.salary

### 6단계 전체 코드
---
파일을 열고 각 사원의 월급을 비교해서 가장 높은 월급을 가진 사원의 정보를 출력한다.

In [11]:
import os

class Employee:
    def __init__ (self, name, division, jobgrade, years, basic):
        self.name = name
        self.division = division
        self.jobgrade = int(jobgrade)
        self.years = int(years)
        self.basic = float(basic)
        self.salary = 0

    def get_name(self):
        return self.name

    def get_division(self):
        return self.division
    
    def get_jobgrade(self):
        return self.jobgrade
    
    def get_years(self):
        return self.years
    
    def get_basic(self):
        return self.basic

    def get_salary(self):
        self.salary = self.basic + (self.basic * self.years * 0.1)
        return self.salary

def make_employee(inputdata):
    name, division, jopgrade, year, basic = inputdata.split(',')
    return Employee(name, division, jopgrade, year, basic)

def summarize():
    file_path = os.getenv("HOME") + '/AIFFEL/Fundamentals/F25_db/data_handling/data/employeefile.txt'
    inputfile = open(file_path, 'r')
    highest = make_employee(inputfile.readline())
    for l in inputfile:
        e = make_employee(l)
        if e.get_salary() > highest.get_salary():
            highest = e
    inputfile.close()
    
    print("The name of the highest salary employee is: ", highest.get_name())
    print("Years of service: ", highest.get_years())
    print("Division: ", highest.get_division())
    print("Salay: ", highest.get_salary())

클래스가 만들어졌다. `summarize()` 메소드를 호출하여 기획의도대로 동작하는지 살펴보자.

In [12]:
summarize()

The name of the highest salary employee is:  Kim YoungJae
Years of service:  7
Division:   Sales
Salay:  1020.0


코드 실행결과를 확인해보자. 잘 나온듯 하다.

그런데, 새로운 사실이 있다. 관리해야 할 데이터 파일이 1개만 있는게 아니라는 사실이다. 여러 파일들에 들어 있는 데이터 항목들도 각각 다른데, 심지어 파일간 데이터들이 서로 연관성이 있어서 여러 개의 파일들을 다루기 위한 코드가 점점 복잡해질 것 같다.

고민하던 차에 당신은 Pandas를 활용하는 것이 도움이 되겠다는 결론에 도달하게 된다.

## 3. 데이터 관리 프로그램 만들기 (2) Pandas와 csv 파일
SQL은 데이터 연산 작업이 가능한, "쿼리(query)를 위한 언어"이다. 파이썬 기반의 프레임 워크인 Pandas를 이용하면 SQL과 유사한 기능을 수행할 수 있다. Pandas를 이용하면 많은 파일들에 흩어져 있는 데이터들을 효과적으로 모아서 깔끔하게 처리할 수 있다.

### 데이터 합치기 : merge, join, concat
---
서로 다른 곳에 있는 데이터를 공통된 값을 사용하여 묶어줄 수 있으면 데이터를 분석하기 매우 쉽다. 그런데 서로 묶을 수 있는 데이터는 어떻게 생긴 데이터일까?

다음과 같은 데이터가 있다고 하자.

* 학생 A: 국어점수 - 90점, 학교 - 고등학교
* 학생 B: 나이 - 12살, 사는 곳 - 서울
* 사원 A: 입사 연도 - 2009년
* 고객 A: 계좌번호 - 11111347234, 카드 소유 여부 - Yes

이런 데이터는 어떻게 합칠 수 있을까? 사람마다 인덱스를 지정해 주어서 순서를 붙이는 걸로 해줄 수밖에 없을 것 같다. 이 데이터는 칼럼(column)을 지정해 줄 수도 없기 때문에 정말 뒤죽박죽 데이터가 될 것 같다.

데이터를 합치는 건 좋지만 그렇다고 막 갖다 붙이면 안될 것이다. 데이터를 합칠 때는 어떤 연관이 있어야 한다. 즉 서로 관계가 있는 데이터에 대해서 합칠 수 있다.

* Pandas에서는 공통으로 연관이 되는 칼럼이 있는 경우에 대해 그 칼럼을 키(key)로 지정해 주어 합치기 연산을 할 수 있다.
* Pandas에서 제공하는 합치기 관련 메소드는 `merge()`, `join()`, `concat()`이 있다.

연산을 하나씩 살펴보자.

#### pd.merge()
아래와 같은 데이터가 있으며, 이들은 서로 다른 테이블에 속해 있다고 하자. 각각을 DataFrame 형태로 표현한다.

In [13]:
import pandas as pd

df1 = pd.DataFrame({'Student': ['KimTaemin','HaJaehwa','JungSayoung','Sonjimin','Leesoomin','KangJun'],
                    'Korean': [90, 85, 88, 35, 40, 44],
                    'English': [80, 90, 40, 44, 55, 90]})
df2 = pd.DataFrame({'Student': ['KimTaemin','HaJaehwa','JungSayoung','Sonjimin','Leesoomin','KangJun'],
                    'Math': [100, 55, 38, 43, 68, 82]})

print(df1)
print('---')
print(df2)

       Student  Korean  English
0    KimTaemin      90       80
1     HaJaehwa      85       90
2  JungSayoung      88       40
3     Sonjimin      35       44
4    Leesoomin      40       55
5      KangJun      44       90
---
       Student  Math
0    KimTaemin   100
1     HaJaehwa    55
2  JungSayoung    38
3     Sonjimin    43
4    Leesoomin    68
5      KangJun    82


`merge()` 메소드는 공통의 칼럼에 있는 값을 키로 사용해 데이터를 합쳐준다. 여기서는 `Student` 가 공통이었기 때문에 자동으로 `Student` 필드의 값이 키가 되어 이를 기준으로 합쳐진다. `merge()` 에는 `on` 인자에 키 값을 넣어 주어 공통 칼럼이 여러 개인 경우를 대비해 직접 값을 넣어줄 수 있으며, 키 값이 될 수 있는 필드가 1개더라도 명확한 코드를 작성하기 위해 키 값을 담은 컬럼(column)의 이름을 직접 나타내주면 좋다.

In [14]:
pd.merge(df1, df2)

,Student,Korean,English,Math
0,KimTaemin,90,80,100
1,HaJaehwa,85,90,55
2,JungSayoung,88,40,38
3,Sonjimin,35,44,43
4,Leesoomin,40,55,68
5,KangJun,44,90,82


In [15]:
pd.merge(df1, df2, on='Student')

,Student,Korean,English,Math
0,KimTaemin,90,80,100
1,HaJaehwa,85,90,55
2,JungSayoung,88,40,38
3,Sonjimin,35,44,43
4,Leesoomin,40,55,68
5,KangJun,44,90,82


#### pd.merge(how='inner')
예제 데이터에 Student 항목은 다 동일한 값이었는데, 겹치는 행이 일부만 있다면 어떻게 해야할까?

In [17]:
df1 = pd.DataFrame({'Student': ['KimTaemin','HaJaehwa','JungSayoung','Sonjimin','Leesoomin','KangJun'],
                    'Korean': [90, 85, 88, 35, 40, 44],
                    'English': [80, 90, 40, 44, 55, 90]})
df2 = pd.DataFrame({'Student': ['Jiyoungmin','KimTaemin'],
                    'Math':[44,33]})

print(df1)
print('---')
print(df2)

       Student  Korean  English
0    KimTaemin      90       80
1     HaJaehwa      85       90
2  JungSayoung      88       40
3     Sonjimin      35       44
4    Leesoomin      40       55
5      KangJun      44       90
---
      Student  Math
0  Jiyoungmin    44
1   KimTaemin    33


공통의 데이터에 대해서만 데이터를 합치는 것을 __inner join__ 이라고 한다. 꼭 Pandas가 아니더라도 관계형 데이터베이스 전반에 쓰이는 용어이다. `merge()`의 `how` 매개변수 기본값은 `"inner"` 이다. 따라서 아무것도 지정해 주지 않으면 inner join으로 연산된다.

In [18]:
pd.merge(df1, df2, how='inner')

,Student,Korean,English,Math
0,KimTaemin,90,80,33


In [19]:
pd.merge(df1, df2)

,Student,Korean,English,Math
0,KimTaemin,90,80,33


#### pd.merge(how='outer')
inner join이 있으면 outer join도 있을 것이다. outer join은 전체 데이터에 합치는 연산을 한다. `how`에 `"outer"` 값을 넘겨주면 된다.

직접 실행하고 결괏값에 어떤 차이가 있는지 살펴보자.

In [20]:
pd.merge(df1, df2, how='outer')

,Student,Korean,English,Math
0,KimTaemin,90.0,80.0,33.0
1,HaJaehwa,85.0,90.0,NaN
2,JungSayoung,88.0,40.0,NaN
3,Sonjimin,35.0,44.0,NaN
4,Leesoomin,40.0,55.0,NaN
5,KangJun,44.0,90.0,NaN
6,Jiyoungmin,NaN,NaN,44.0


#### df.join()
`merge()` 대신 `join()` 메소드를 이용할 수도 있다. `join()` 은 `DataFrame` 클래스의 메소드이다.

In [21]:
df1.join(df2, how='outer', lsuffix='_caller', rsuffix='_other')   # '_caller'인 df1 컬럼이 왼쪽에 가도록 배치

,Student_caller,Korean,English,Student_other,Math
0,KimTaemin,90,80,Jiyoungmin,44.0
1,HaJaehwa,85,90,KimTaemin,33.0
2,JungSayoung,88,40,NaN,NaN
3,Sonjimin,35,44,NaN,NaN
4,Leesoomin,40,55,NaN,NaN
5,KangJun,44,90,NaN,NaN


#### df.concat()
데이터를 합치는 또 다른 방법으로 이어 붙이기가 있다. 적층 또는 연결이라고 표현하기도 한다. 데이터의 칼럼을 늘리는 것이 아니라 데이터의 행을 합치는 것이다.

In [22]:
df1 = pd.DataFrame({'Student': ['KimTaemin','HaJaehwa','JungSayoung','Sonjimin','Leesoomin','KangJun'],
                    'Korean': [90, 85, 88, 35, 40, 44],
                    'English': [80, 90, 40, 44, 55, 90]})
df2 = pd.DataFrame({'Student': ['Jiyoungmin','LeeJae','KimJaehee'],
                    'Korean': [44,73,100]})

print(df1)
print(df2)

       Student  Korean  English
0    KimTaemin      90       80
1     HaJaehwa      85       90
2  JungSayoung      88       40
3     Sonjimin      35       44
4    Leesoomin      40       55
5      KangJun      44       90
      Student  Korean
0  Jiyoungmin      44
1      LeeJae      73
2   KimJaehee     100


In [23]:
pd.concat([df1,df2], sort=False)

,Student,Korean,English
0,KimTaemin,90,80.0
1,HaJaehwa,85,90.0
2,JungSayoung,88,40.0
3,Sonjimin,35,44.0
4,Leesoomin,40,55.0
5,KangJun,44,90.0
0,Jiyoungmin,44,NaN
1,LeeJae,73,NaN
2,KimJaehee,100,NaN


In [24]:
pd.concat([df1, df2], sort=False, ignore_index=True)

,Student,Korean,English
0,KimTaemin,90,80.0
1,HaJaehwa,85,90.0
2,JungSayoung,88,40.0
3,Sonjimin,35,44.0
4,Leesoomin,40,55.0
5,KangJun,44,90.0
6,Jiyoungmin,44,NaN
7,LeeJae,73,NaN
8,KimJaehee,100,NaN


인덱스가 처음부터 시작하는 경우가 있는데 그때는 `ignore_index` 인자를 `True`로 설정해준다.

Pandas의 주요 메소드 `merge()`, `join()`, `concat()` 를 사용해 데이터를 합치는 방법에 대해서 살펴보았다.

데이터를 합치는 __join__ 연산은 데이터 연산에 있어서 굉장히 중요한 개념이다. 위에서는 join 연산을 Pandas의 `merge()`와 `join()`으로 구현해 보았는데, 데이터 연산용 언어인 SQL에서도 비슷한 단어와 의미로 표현한다.

<img src="./image/sqljoin.png" alt="sqljoin" />

## 4. 데이터 관리 프로그램 만들기 (3) Pandas의 유용한 기능들
Pandas를 사용할 경우, 그렇지 않았다면 매번 힘들게 구현해야 했을 많은 기능을 Pandas를 통해 손쉽게 얻을 수 있다. Pandas가 내부적으로 인덱스(index)를 가지고 있어서 필터링, 탐색, 그룹 질의 등을 매우 빠르게 수행해 주기 때문이다.

### 필터링 연산
---
이번에는 조건에 따라 특정 데이터를 선택하는 방법, 즉 데이터 필터링 연산에 대해 알아보자.

필터링 방법에는 몇 가지 방법이 있다. 경우에 따라 몇 개의 행들을 먼저 고르고 거기서 원하는 컬럼만 선택하거나, 혹은 컬럼을 먼저 선택한 다음 특정 행만 선택하는 순서로 데이터를 필터링 할 수도 있다.

아래에서 실험해 보기 위한 데이터 프레임을 먼저 만들자.

In [25]:
df = pd.DataFrame({"A": [1,4,7], "B": [2,5,8], "C":[3,6,9]})

df

,A,B,C
0,1,2,3
1,4,5,6
2,7,8,9


#### 1. df['컬럼 명'] 형식 사용
데이터 프레임에 대괄호 기호를 쓰고 컬럼 명을 입력한다.

In [26]:
df['A']

0    1
1    4
2    7
Name: A, dtype: int64

#### 2. loc() 사용
`loc()`은 라벨 을 사용하여 행 또는 열(컬럼)을 지정, 데이터를 추출한다. 사용 형식은 다음의 식으로 기억하자.

```python
df.loc[[행],[열]]
```

아래부터는 살짝 헷갈릴 수 있으니 하나씩 실행하고, 결괏값이 왜 그렇게 나왔는지 차근차근 고민하고 이해하자.



In [27]:
print(df.loc[0])

A    1
B    2
C    3
Name: 0, dtype: int64


In [28]:
print(df.loc[0, 'B'])

2


loc는 리스트의 슬라이스 연산과 비슷한 부분도 있다.

In [29]:
print(df.loc[:, 'A'])

0    1
1    4
2    7
Name: A, dtype: int64


#### 3. iloc() 사용
`iloc()`은 정수 인덱스 를 사용하여 행 또는 열(컬럼)을 지정, 데이터를 추출한다.

```python
df.iloc[[행],[열]]
```

마찬가지로 살짝 헷갈릴 수 있으니 하나씩 실행하고, 결괏값이 왜 그렇게 나왔는지 차근차근 고민하고 이해해야한다. 위에서 나온 `loc()` 과도 비교해보자.

In [30]:
print(df.iloc[0])

A    1
B    2
C    3
Name: 0, dtype: int64


In [31]:
print(df.iloc[:, 0])

0    1
1    4
2    7
Name: A, dtype: int64


`iloc()`에 컬럼 이름을 주면 어떻게 될까?

In [32]:
print(df.iloc[0, 'B'])

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

에러가 난다. 의도를 추측해 고쳐보건대, 다음처럼 사용해야 한다.

In [33]:
print(df.iloc[0, 1])

2


### 그룹연산: `groupby()`, `apply()`
---
값을 선택하거나 데이터를 정렬할 때 기준이 되는 값을 키(key)라고 부른다. 일반적인 데이터베이스에서는 이 키 값에 따라 그룹을 묶은(grouping) 뒤 원하는 연산을 수행할 수 있다. Pandas에서는 `groupby()` 연산이라고 한다.

#### groupby() 연산 메커니즘

In [34]:
df = pd.DataFrame({'Columns1':['A','A','B','B','C','C','A','B'],
                   'Columns2':[10, 2, 30, -6, 8, 9, 5, 2]})

df

,Columns1,Columns2
0,A,10
1,A,2
2,B,30
3,B,-6
4,C,8
5,C,9
6,A,5
7,B,2


* groupby()객체를 생성
* groupby()객체의 연산을 수행
    - max(), min(), sum(), mean()
    - +apply() 메소드를 통해 특수 수식에 대한 연산을 수행할 수 있다.

#### apply()

In [36]:
import numpy as  np
df.groupby(['Columns1']).max().apply(np.sqrt)

,Columns2
Columns1,
A,3.162278
B,5.477226
C,3.000000


## 5. 데이터 관리 프로그램 만들기 (4) Pandas Transform 실전 연습
Pandas의 강력하고 유용한 기능들을 통해 실전형 데이터 관리 프로그램 구현에 한 발 더 다가서게 되었다. 아래 문제는 [Anaconda의 SF 임금 예제](https://anaconda.org/gwinnen/sf-salaries-exercise/notebook)를 한글로 번역한 것이다. 직원 임금 관리 프로그램을 제작한다면 필요한 예제를 통해 Pandas 활용의 전문가가 되어보자.

### 데이터
---
문제를 풀기 위해 데이터를 불러오자

In [37]:
import pandas as pd
import numpy as np

file_path = os.getenv("HOME") + '/AIFFEL/Fundamentals/F25_db/data_handling/data/Salaries.csv'
sal = pd.read_csv(file_path)

sal.head(5)

/home/aiffel-dj10/anaconda3/envs/aiffel/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


### 문제
---

In [38]:
sal['BasePay'].mean()

66327.68895967308

* 평균 basepay는 66327.68895967308이다

In [39]:
sal['OvertimePay'].max()

245131.88

* 최대 OvertimePay는 245131.88이다.

In [40]:
sal[sal['EmployeeName'] == 'JOSEPH DRISCOLL']['JobTitle']

24    CAPTAIN, FIRE SUPPRESSION
Name: JobTitle, dtype: object

* JOSEPH DRISCOLL의 job title은 CAPTAIN, FIRE SUPPRESSION이다

In [41]:
sal[sal['EmployeeName'] == 'JOSEPH DRISCOLL']['TotalPayBenefits']

24    270324.91
Name: TotalPayBenefits, dtype: float64

* JOSEPH DRISCOLL의 급여는 270324.91이다

In [42]:
sal[sal['TotalPayBenefits']==sal['TotalPayBenefits'].max()]['EmployeeName']

0    NATHANIEL FORD
Name: EmployeeName, dtype: object

* 월급(Benefit포함) 이 가장 높은 사람은 NATHANIEL FORD이다.

In [43]:
sal[sal['TotalPayBenefits']==sal['TotalPayBenefits'].min()]

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
110530,110531,David P Kucia,Police Officer 3,NaN,0.0,0.0,-33.89,0.0,-33.89,2013,NaN,San Francisco,NaN


* 월급(Benefit포함) 이 가장 낮은 사람은 David P Kucia이다.

In [44]:
sal.groupby('Year').mean()['BasePay']

Year
2011    63595.956517
2012    65436.406857
2013    69630.030216
2014    66573.154204
Name: BasePay, dtype: float64

2011~2014년 동안의 연도별 BasePay의 평균은 각각 
* 2011    63595.956517
* 2012    65436.406857
* 2013    69630.030216
* 2014    66573.154204

이다.

In [45]:
sal['JobTitle'].nunique()

2158

* Job Title의 종류는 2158개이다.

In [46]:
sal['JobTitle'].value_counts().head(5)

Transit Operator                7036
Special Nurse                   4389
Registered Nurse                3736
Public Svc Aide-Public Works    2518
Police Officer 3                2421
Name: JobTitle, dtype: int64

* Job Title중 가장 많은 상위 5개의 Job Title은 Transit Operator, Special Nurse, Registered Nurse, Public Svc Aide-Public Works, Police Officer 3 이다.

In [48]:
def chief_string(title):
    if 'chief' in title.lower().split(): 
        return True
    else:
        return False
sum(sal['JobTitle'].apply(lambda x : chief_string(x)))

477

* Chief 란 단어를 갖는 Job Title은 477개이다.

## 6. 데이터베이스 (1) 다중 사용자 환경
현실 세계에서는 데이터를 어떻게 관리할까? 은행을 예로 들어보자. 대형 은행은 약 1,000만 명 인원의 계좌 정보를 가지고 있다. 그리고 인원당 하나의 계좌를 가지란 보장이 없기 때문에 계좌번호만 생각해도 이 데이터 수는 2,000만..3,000만… 그리고 각 계좌당 거래 정보들까지 하면… 엄청 많아질 것이다. 게다가 엄청나게 많은 사용자가 동시 접속해서 계좌입출금 등 다양한 업무를 수행할 것이다.

이런 데이터의 영속성을 어떻게 관리하면 좋을까? 이전 스텝까지 다루었던 csv 파일 시스템과 Pandas만으로 우리는 이런 데이터를 잘 다루어 낼 수 있을까?

A 고객과 B 고객이 거의 동시에 C 고객에게 입금을 했을 때의 아래와 같은 시나리오를 생각해보자.

In [49]:
C_account = 100      # 현재 C 고객 계좌 잔고는 100

C_account_by_A_process = 100    # A 고객 응대 프로그램이 C 고객 계좌 잔고를 조회했습니다 
C_account_by_B_process = 100    # B 고객 응대 프로그램이 C 고객 계좌 잔고를 조회했습니다 

C_account_by_A_process = C_account_by_A_process + 50   # A 고객이 50을 입금했습니다. 
C_account_by_B_process = C_account_by_B_process + 30   # A 고객이 30을 입금했습니다. 

C_account = C_account_by_A_process     # A 고객의 입금이 C 고객 계좌 잔고에 반영되었습니다. 
C_account = C_account_by_B_process     # B 고객의 입금이 C 고객 계좌 잔고에 반영되었습니다. 

print(C_account)     # C 고객 계좌 잔고는 얼마일까요?

130


지금까지 잘 사용해 온 Pandas 기반 프로그램은 C 고객의 잔고를 130이라고 기억하고 있을 가능성이 높다. 왜 그럴까? 이것은 파일시스템만으로 데이터의 영속성이 잘 관리되기 어렵기 때문이다. 위 시나리오의 가장 큰 문제점은 트랜잭션(Transaction)의 개념이 없다는 점이다. 동일한 데이터를 여러 프로세스가 동시에 접근해서 변경하려고 들 때, 한 프로세스의 변경이 다른 프로세스의 변경을 무효화해서는 안 된다.

트랜잭션의 개념에 대해서 다음 문서 참고.<br>
(참고) [트랜잭션이란 무엇인가?](https://coding-factory.tistory.com/226)

이제 다수 사용자에 대응할 수 있는 실시간 트랜잭션 처리 기능을 갖춘, 데이터의 정합성을 보장하는 훌륭한 데이터 관리 프로세스를 필요로 하게 되었다. 그래서 필요한 것이 바로 __데이터베이스 관리 시스템(DBMS)__이다.

데이터 사용량은 2007년 스마트폰의 도입과 함께 SNS 서비스가 인기를 얻으며 폭발적으로 늘어났다. 이런 시대를 빅데이터(big data) 시대라고 한다. 데이터를 저장하고 관리하는 기법 또한 매우 빠른 속도로 발전하고 있다. 그런데 빅데이터 이전에는 데이터가 없었던 걸까? 빅데이터 이전에도 우리는 일반 회사의 실험 자료들, 금융, 회계, 거래 내역, 사진 등 많은 데이터를 컴퓨터에 보관해왔다. 데이터를 어떻게 저장하고 관리할지는 빅데이터 시대 이전부터 우리 생활과 아주 밀접하게 관련된 문제였던 것이다. 데이터베이스의 역사는 매우 오래되었다. 좀 더 넓고 오래된 분야에서의 데이터 관리에 대해 다루어 보겠다.

## 7. 데이터베이스 (2) 데이터베이스의 세계로
정보를 저장하고 기록하는 데이터 관리를 거슬러 올라가 보면 태초의 상형문자가 그 기원이 아닐까 생각된다.

현재 우리가 데이터베이스라고 말하는 시스템은 컴퓨터가 개발된 1960년대부터 시작된 개념이다. (처음에는 "파일 시스템"으로 불렸다) 현재 많이 사용되고 있는 관계형 데이터베이스(RDB: relational database)는 1970년대 Ted Codd에 의해 최초로 고안되었다. (아래 유튜브 영상에서 썸네일에 나온 사람이다)

아래 동영상을 시청해보자.

[![데이터베이스](http://img.youtube.com/vi/KG-mqHoXOXY/0.jpg)](https://youtu.be/KG-mqHoXOXY) 

### 데이터베이스, 서버, 데이터만 관리하는 컴퓨터
---
앞의 예제에서는 각 직원에 대한 데이터를 파일로부터 한 줄씩 후 클래스로 만든 뒤, 코딩으로 가장 높은 월급을 계산하는 프로그램을 만들었다.


<img src="./image/db2.png" alt="DB" />

처음에는 이렇게 데이터를 관리하고 연산하는 작업을 모두 하나의 컴퓨터에서 실행했다. 그런데 실무에서의 데이터는 굉장히 복잡하다. 사원의 정보가 정말 텍스트 파일 하나에 모두 담길 수 있을까? 대규모 회사라면 그 파일의 크기가 어마어마할 것이다. 한 개 회사의 직원 데이터를 넘어 국가 통계청의 데이터, 은행의 계좌 관리 데이터 등 주변을 조금만 둘러보아도 거대한 규모의 데이터베이스가 많다는 사실을 알 수 있다. 이때 데이터를 만들고, 읽고, 쓰는 작업을 여러 명이 모두 하나의 컴퓨터에 하는 건 매우 힘든 일이다.

그래서 데이터만을 저장하는 공간을 하나 만든다. 흔히 물리적 컴퓨터에 방점을 두어 데이터 서버 컴퓨터, 또는 추상적인 정보의 집합에 방점을 두어 데이터베이스라고 한다.

구조화해보면 아래 block diagram과 같다. 데이터만을 위한 데이터 서버 컴퓨터가 있고, 해당 서버에 접근하는 전용 프로그램을 이용해서 데이터를 읽고 수정하는 것이다. 그리고 이 접근, 수정 등의 일련의 작업은 일관된 방법과 형식을 사용하면 편리할 것이다.

이렇게 데이터를 요청하는 "쿼리(query)"를 작성하기 위한 언어 중 하나가 SQL(Structured Query Language)이다. SQL은 영어에서 볼 수 있듯이 구조적 질의어(=데이터 서버에게 원하는 정보를 일목요연하게 말해주는 것)를 뜻하며, 일종의 프로그래밍 언어로 발전되어 왔다.

<img src="./image/sql.png" alt="SQL" />

### 소프트웨어의 요소와 소프트웨어를 만드는 사람들
---
각 서버를 만드는 일이 어떻게 분업되어 있는지 은행 거래 시스템을 예로 생각해보자.

이 서비스의 개발팀은 데이터 서버 컴퓨터를 따로 두어 데이터를 관리해야 하고, 사용자가 조작할 수 있는 (계좌번호 입력창 등) 화면을 만들고, 사용자가 데이터를 조회하고 거래도 할 수 있게 만들어야 한다. 그리고 이 모든 거래 정보는 데이터 서버 컴퓨터에 저장되어야 할 것이다.

<img src="./image/db3.png" alt="DB" />

사용자는 소프트웨어 어플리케이션을 설치해서 사용한다. 편의상 "어플리케이션"이 사용자가 직접 사용하는 프로그램(프론트엔드)과 뒤에서 이 프로그램을 지원해주기 위한 서버(백엔드)를 모두 통칭한다고 가정하자. 통상 개발자라고 하면 이 소프트웨어 어플리케이션을 개발하는 사람들을 뜻한다. 그리고 DBA(Database Architect)는 데이터를 어떻게 관리할지, 어떤 항목으로 관리할지 등을 설계하고 데이터베이스에 접근하기 위한 쿼리를 작성한다. 이때 설계하는 데이터베이스의 구조를 전문 용어로 __스키마(schema)__라고 부른다.